In [21]:
from serpapi.google_search import GoogleSearch

def search_google_news(keyword):
    print(f"Keyword: {keyword}")
    search = GoogleSearch({
        "q": keyword,
        "tmb": "nws",
        "safe": "active",
        "api_key": open("serp_api_key.txt").read().strip()
    })
    result = search.get_dict()
    news_results = [item['link'] for item in result['news_results']]
    return news_results

In [23]:
import autogen
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model":["gpt-4o-mini"],
    },
)
gpt_api_key = config_list[0]["api_key"]

In [24]:
llm_config={
    "request_timeout":600,
    "seed":42,
    "config_list":config_list,
    "temperature":0,
    "functions":[
        {
            "name":"search_google_news",
            "description":"Search google news by keyword",
            "parameters": {
                "type":"object",
                "properties":{
                    "keyword":{
                        "type":"string",
                        "description":"The keyword that is used to search google news",
                    }
                },
                "required":["keyword"],
            },
        }
    ],
}

In [25]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

uesr_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,#避免无限对话循环
    code_execution_config={"work_dir":"."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"search_google_news":search_google_news}
)

In [26]:
uesr_proxy.initiate_chat(
    assistant,
    message="""
Search google news in the topic of "Hamas", and write a news based on them.
You should avoid bullet points.
Instead, use more expressive words to form a news that's like one written by a well recognized journalist.

Start the work now.
"""
)

user_proxy (to assistant):


Search google news in the topic of "Hamas", and write a news based on them.
You should avoid bullet points.
Instead, use more expressive words to form a news that's like one written by a well recognized journalist.

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: search_google_news *****
Arguments: 
{"keyword":"Hamas"}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_google_news...
Keyword: Hamas
user_proxy (to assistant):

***** Response from calling function "search_google_news" *****
Error: 'news_results'
***************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: search_google